In [1]:
import minsearch

In [2]:
import json

In [3]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [4]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [5]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [6]:
index = minsearch.Index(
    text_fields = ["question","text","section"],
    keyword_fields = ["course"]
)

In [7]:
index.fit(documents)

In [8]:
query = 'How do I execute a command in a running docker container?'

In [18]:
boost = {'question': 4.0, 'section':0.5}

results = index.search(
    query=query,
    filter_dict={'course': 'machine-learning-zoomcamp'},
    boost_dict=boost,
    num_results=5
)

In [10]:
# results

In [11]:
from openai import OpenAI

In [12]:
client = OpenAI()

In [13]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user","content":prompt}]
    )
    return response.choices[0].message.content

In [19]:
llm(query)

'To execute a command in a running Docker container, you can use the `docker exec` command. This allows you to run commands inside an existing, running container. Below are the general steps and examples for executing commands in a Docker container:\n\n### Step-by-Step Instructions:\n\n1. **Identify the Container ID or Name:**\n   First, you need to know either the `CONTAINER ID` or the `name` of the container you want to access. You can list all running containers with:\n   ```sh\n   docker ps\n   ```\n   This will display a list of all currently running containers along with their IDs and names.\n\n2. **Execute a Command:**\n   Use the `docker exec` command to run a command inside the container. The basic syntax is:\n   ```sh\n   docker exec [OPTIONS] CONTAINER COMMAND [ARG...]\n   ```\n   - Replace `CONTAINER` with the container ID or name.\n   - Replace `COMMAND` with the command you wish to run.\n   - `[OPTIONS]` can be various flags you want to pass (optional).\n\n3. **Examples:*

In [20]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

Q: {question}

{context}
""".strip()

In [21]:
context = ""

for doc in results:
    context = context + f"question: {doc['question']}\nanswer: {doc['text']}\n\n" 

In [23]:
prompt = prompt_template.format(question=query, context=context).strip()

In [24]:
len(prompt)

2462

In [28]:
# !pip install tiktoken
import tiktoken

In [29]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [37]:
tokens = encoding.encode(prompt)
print(len(tokens))

529
